In [1]:
import pandas as pd
import random
import string
import pulp
from itertools import combinations

In [2]:
num_members = 20  # 人数
num_departments = 5  # 部署数
num_groups = 5  # いくつのグループに分けるか
num_trial = 10  # 何回グルーピングを行うか

random.seed(0)

ID = list(map(str, range(1, num_members + 1)))
DEP = random.choices(string.ascii_uppercase[:num_departments], k=num_members)

id2dep = {}
dep2id = {}
for i, d in zip(ID, DEP):
    id2dep[i] = d
    if d not in dep2id:
        dep2id[d] = []
    dep2id[d].append(i)

In [4]:
G = list(map(str, range(1, num_groups + 1)))

# 同じグループになった回数
cnt = {}
for i1, i2 in combinations(ID, 2):
    cnt[i1, i2] = 0

for t in range(1, num_trial + 1):
    print(f"Trial #{t}")

    # 人×グループの全組み合わせ
    comb = [(i, g) for i in ID for g in G]
    x = pulp.LpVariable.dicts("x", comb, cat="Binary")

    prob = pulp.LpProblem("Assignment", pulp.LpMinimize)

    # 各人は1つのグループに所属
    for i in ID:
        prob += pulp.lpSum([x[i, g] for g in G]) == 1

    # 各グループの人数の制約
    min_group = int(num_members / num_groups)
    max_group = int(num_members / num_groups + 0.999999)
    if min_group == max_group:  # 割り切れるケース
        for g in G:
            prob += pulp.lpSum([x[i, g] for i in ID]) == min_group
    else:  # 割り切れないケース
        for g in G:
            prob += pulp.lpSum([x[i, g] for i in ID]) >= min_group
            prob += pulp.lpSum([x[i, g] for i in ID]) <= max_group
    
    # 部署がばらけるように（甘めの制約）
    for g in G:
        for d in set(DEP):
            up = int(len(dep2id[d]) / num_groups) + 1  # 1グループ内同部署数の上限
            prob += pulp.lpSum([x[i, g] for i in dep2id[d]]) <= up
    
    # 補助変数tを導入して同じグループになったペナルティを計算
    t = pulp.LpVariable("t", lowBound=0, cat="Continuous")
    for g in G:
        for i1, i2 in combinations(ID, 2):
            prob += (x[i1, g] + x[i2, g] -1) * cnt[i1, i2] <= t

    prob += pulp.lpSum(t)  # ペナルティの最大値を最小化
    
    status = prob.solve(pulp.PULP_CBC_CMD(msg=0))
    print(f"  status:{status}")
    for g in G:
        gm = [i for i in ID if x[i, g].value() == 1]
        dic_dep = {d: 0 for d in sorted(set(DEP))}
        for i in ID:
            if x[i, g].value() == 1:
                dic_dep[id2dep[i]] += 1
        print(f"  Group {g}: {gm}, {dic_dep}")

        for i1, i2 in combinations(gm, 2):
            cnt[i1, i2] += 1

for i1, i2 in combinations(ID, 2):
    print(f"{i1}-{i2}: {cnt[i1, i2]}")

Trial #1
  status:1
  Group 1: ['7', '8', '10', '12'], {'B': 1, 'C': 2, 'D': 1, 'E': 0}
  Group 2: ['4', '14', '18', '19'], {'B': 1, 'C': 0, 'D': 1, 'E': 2}
  Group 3: ['6', '9', '11', '15'], {'B': 0, 'C': 2, 'D': 1, 'E': 1}
  Group 4: ['1', '5', '16', '20'], {'B': 1, 'C': 1, 'D': 0, 'E': 2}
  Group 5: ['2', '3', '13', '17'], {'B': 1, 'C': 1, 'D': 1, 'E': 1}
Trial #2
  status:1
  Group 1: ['1', '2', '4', '6'], {'B': 1, 'C': 1, 'D': 1, 'E': 1}
  Group 2: ['5', '8', '9', '17'], {'B': 1, 'C': 2, 'D': 0, 'E': 1}
  Group 3: ['7', '11', '13', '18'], {'B': 1, 'C': 0, 'D': 1, 'E': 2}
  Group 4: ['10', '15', '16', '19'], {'B': 1, 'C': 1, 'D': 1, 'E': 1}
  Group 5: ['3', '12', '14', '20'], {'B': 0, 'C': 2, 'D': 1, 'E': 1}
Trial #3
  status:1
  Group 1: ['2', '8', '19', '20'], {'B': 1, 'C': 0, 'D': 1, 'E': 2}
  Group 2: ['3', '4', '7', '9'], {'B': 1, 'C': 2, 'D': 1, 'E': 0}
  Group 3: ['1', '12', '13', '15'], {'B': 1, 'C': 1, 'D': 1, 'E': 1}
  Group 4: ['6', '16', '17', '18'], {'B': 1, 'C': 1, 'D